1. ACG (A Cloud Guru) code-server setup
   <br>ACG cloud code serve enable end user to use any web browser to access visio studio IDE (vscode), using remote extension will make dev team to using un

   ![ACG Code Server Arch](./ACG-code-server-arch.jpg)
   ![Vscode Remote Extension](./vscode-remote-extension.jpg)
   - create code server
      - On ACG select "cloud playground", click "New Server" filled as shown bellow

         ![New Server](./ACG-new-server.jpg)
         <br>select 4 units, 2vCPU, 8G Mem as host VM, which can be used for container lab in future 
   
      - Newly created code server will provide you with public access ip address & initial password

         ![Code Server](./code-server.jpg)
         <br>code server public ip: 44.205.4.135, temporary password:AGXP3443ycfi

      - Using termainal ssh to code server with
         <br>public ip: 44.205.4.134 
         <br>username: cloud_user 
         <br>temporary password: AGXP3443ycfi
         <br>update with new password: cl0uds4ndb0x2022!

         ![Code Server SSH](./code-server-SSH.jpg)
         <br>update code server with user selected password

   - Dry-run google cloud code server
      - download google code server from https://code-server.dev website

         ![Google Code Server](./code-server-curl.jpg)
         <br>curl -fsSL https://code-server.dev/install.sh | sh -s -- --dry-run
               
   - download google cloud code server
      - install code server with curl
      
         ![code Server Install](./code-server-curlinstall.jpg)
         <br>curl -fsSL https://code-server.dev/install.sh | sh

      - start code-server as a background process

         ![Code Server Start](./code-server-start.jpg)
         <br>sudo systemctl enable --now code-server@$USER

      - config code server with changed password

         ![Code Server Config YAML](./code-server-configYaml.jpg)
         <br>--- vi editing---<br>
         ![Code Server Default Config](./code-server-configYaml-password.jpg)
         <br>update into<br>
         ![Code Server Change Password](./code-server-configYaml-password1.jpg)

         <br>--- restart code server service ---<br>
         ![Code Server Restart Service](./code-server-restart.jpg)
         <br>sudo systemctl restart code-server@cloud_user.service

         - debug code server ICMP Reject Issue
         
            ![Code Server Iptables](./code-server-iptables.jpg)
            <br>sudo iptables -L

            ![Code Server Watch Iptables](./code-server-iptables-watch.jpg)
            <br>sudo watch -d iptables -L -v -n

            ![Code Server Iptables status](./code-server-iptables-status.jpg)

            ![Coder Server Iptables-save](./code-server-iptables-save.jpg)
            <br>sudo iptables-save

            ![Code Server Iptables Delete](./code-server-iptables-Delete.jpg)
            <br>sudo iptables -D LIBVIRT_FWO -i virbr0 -j REJECT --reject-with icmp-port-unreachable
            <br>sudo iptables -D LIBVIRT_FWI -o virbr0 -j REJECT --reject-with icmp-port-unreachable

            ![Code Server Libvirt Util](./code-server-libvirt.jpg)
            <br>sudo yum install libvirt-client

            ![Code Server Libvirt Net-edit](./code-server-libvirt-net-edit.jpg)
            <br>sudo virsh net-edit default

            ![Code Server Libvirt Default](./code-server-libvirt-default-nat.jpg)

            ![Code Server Libvirt Nutshell](./code-server-libvirt-nutshell.jpg)
            <br>libvirt nat mode. The libvirt daemon puts this configuration into effect when it starts up, 
            <br>so if you have the libvirt daemon set to start automatically on each boot it should always be present.

            ![Code Server Libvirt Routed Mode](./code-server-libvirt-routed.jpg)
            <br>libvirt routed mode. Hosts in DMZ provide services both to LAN hosts and WAN. Therefore, they need to be
            <br> accessible by other computers on the intranet and also by computers in the internet

      - config code server to use https service with port 8443

         ![Code Server Netstat](./code-server-netstat.jpg)
         <br>sudo netstat -anpe

         - edit code server config.yaml file to use nat/forwarding, port 443 
         <br>(note: code server initial boot up bind to localhost 127.0.0.1, change to internet 0.0.0.0,  443 port used by 
             https, the final port will be use 8443)
            ![Code Server Nat Forwarding](./code-server-configYAML-edit.jpg)
            <br>sed -i.bak 's/cert: false/cert: true/' ~/.config/code-server/config.yaml
            <br>sed -i.bak 's/bind-addr: 0.0.0.0:443/bind-addr: 0.0.0.0:8443/' ~/.config/code-server/config.yaml
            <br>sed -i.bak 's/auth: password/auth: none/' ~/.config/code-server/config.yaml

         - add http service into firewall
         
            ![Code Server Firewall](./code-server-firewall.jpg)
            <br>sudo firewall-cmd --zone=public --list-services
            <br>sudo firewall-cmd --zone=public --add-service=http
            <br>sudo firewall-cmd --list-all
            <br>sudo firewall-cmd --get-active-zone
            <br>sudo firewall-cmd --get-default-zone
            <br>sudo firewall-cmd --list-all-zones

         - add port 8443 into firewall rules

            ![Code Server Firewall Port](./code-server-firewall-cmd-port.jpg)
            <br>sudo firewall-cmd --zone=public --add-port=8443/tcp

            ![Code Server Firewall Port Add](./code-server-firewall-cmd-port-add.jpg)
            
            ![Code Server Netstat](./code-server-netstat1.jpg)
            <br>sudo netstat -anpe
            
         - access vscode from browser

            ![Code Server Browser](./code-server-browser.jpg)

   - build python dev environment
      - penv
         <br>sudo yum install openssl-devel -y
         <br>sudo curl https://pyenv.run | bash



      - important environment variable

         - Load pyenv automatically by appending the following to ~/.bash_profile if it exists, 
         otherwise ~/.profile (for login shells) ~/.bashrc (for interactive shells) :

            export PYENV_ROOT="$HOME/.pyenv"  
            command -v pyenv >/dev/null || export PATH="$PYENV_ROOT/bin:$PATH"  
            eval "$(pyenv init --path)"  
            eval "$(pyenv init -)"

         - Restart your shell for the changes to take effect.

            source ~/.bashrc

         - Load pyenv-virtualenv automatically by adding the following to ~/.bashrc:

            eval "$(pyenv virtualenv-init -)"

      - install python 3.10
         <br>pyenv install 3.10
         <br>pip3.10 install --upgrade pip
         <br>


2. vscode local IDE remote ssh + remote deploy

   ![vscode IDE](./vscode-IDE.jpg)
   - remote ssh 
      - open command pallete using CTRL + SHIFT + P, in search bar type "remote ssh", selete "Remote-SSH Add Host"
       or "Remote-SSH Connect to" accorgingly, enter password of the remote code server with cloud_user as username

         ![vscode remote ssh](./vscode-remote-ssh.jpg)  
         ![vscode remote ssh password](./vscode-remote-ssh-password.jpg)  
         ![vscode remote ssh local config1](./vscode-remote-ssh-config1.jpg)  
         ![vscode remote ssh local config2](./vscode-remote-ssh-config2.jpg)  
         ![vscode remote ssh output](./vscode-remote-ssh-output.jpg)  
         

   - remote interpreter
     Microsoft vscode lack to support of remote interpreter, which is the major reason people pursue pycharm as IDE.  The
     following ticket has been open for the support of this feature. 
     https://github.com/Microsoft/vscode-python/issues/79
      - Remote server has a fully configured Python environment, which is more or less identical to the deployment setup.
      - The developer workstation may have Python installed, but not necessary fully configured environment. 
      - It is always preferable to development using an environment that is similar or as deployment environment
      - The development and deployment environments may even be running different operating systems.  

      ![vscode task](./vscode-task01.jpg)
      ![vscode task creation](./vscode-task02.jpg)
      ![vscode task rsync](./vscode-task03.jpg) 
     
     
   - remote deployment(code sync) - remote deployment is used for using local folder + local interpreter to compile, and then
     upload into remote folder
      - vscode can use open folder (on remote DEV server), or use sftp / ftp to deploy local folders / files to remote
        folders / files. Here is an example of using Pro Deployer - SFTP / FTP

        ![vscode remote ssh deployment](./vscode-remote-ssh-deployExtension.jpg)


3. pycharm local IDE remote ssh + remote deploy

   ![pycharm IDE](./pycharm-IDE.jpg)
   - remote ssh 
      - open setting using CTRL + ALT + S, under "Project" select "Python Interpreter". In the pop menu, click "Gear" icon 
        and select "Add" for configuring remote-ssh. Filling remote code-server username/password/port info

        ![pycharm remote ssh](./pycharm-remote-ssh1.jpg)  
        ![pycharm remote ssh setup](./pycharm-remote-ssh2.jpg)
   

   - remote deployment
      - open setting using CTRL + ALT + S, under "Build" select "Deployment". On opened panel click "+" icon and choose "SFTP"
       
        ![pycharm remote deployment](./pycharm-remote-deployment.jpg)

      - on "SSH Configuration" box, click "..." to add SSH connection details: username/password, then test connection

        ![pycharm remote deployment ssh](./pycharm-remote-deployment1.jpg)
        ![pycharm remote deployment ssh status](./pycharm-remote-deployment2.jpg)

      - once SSh connection verified success, click "Autodetect" to finish remote coder server directory rendering, adding
        remote code server web URL details

        ![pycharm remote deployment ssh directory](./pycharm-remote-deployment3.jpg)

      - select "Mapping" tab, from "local path" click "book" icon to add local dev codespace directory, from "Deployment path"
        click "book" icon to choose remote code server remote dev codespace directory.

        ![pycharm remote deployment mapping](./pycharm-remote-deployment-mapping-directory.jpg)
        ![pycharm remote deployment mapping dir](./pycharm-remote-deployment-mapping-directory.jpg)  


- vscode useful shortcut cheat sheet
   - integrated terminal
      ```
      Show integrated terminal            Ctrl + `  
      User Settings                       CTRL + ,  
      Open Command Palette                Ctrl + Shift + P  
      Indent multiple lines               Tab   (insert mode)  
      Unindent multiple lines             Shift + Tab (insert mode)  
      Comment Line / Block                
      Select Code Block(to block begin)   
      Select Code Block(to block end)     

      ```

- pycharm useful shortcut cheat sheet
   - integrated terminal
      ```
      Show integrated terminal            ALT + F12, ALT + T  
      User Settings                       CTRL + ALT + S  

      Indent multiple lines               CTRL + I (insert mode)
      Unindent multiple lines             
      Comment Line / Block                CTRL + /  
      Select Code Block(to block begin)   CTRL + SHIFT + [
      Select Code Block(to block end)     CTRL + SHIFT + ]
      ```


   